# Set references for v7 tuning and export to ONNX

This notebook is dedicated to apply the linear correction in the neural network output w.r.t the avgmu and export the v9 best models to ONNX/keras format. Usually, keras versions is used into the prometheus framework. The ONNX version will be used into the athena framework.

**NOTE**: ONNX is a Microsoft API for inference.

**NOTE**: We will export all tunings from v9 r0 derivation

In [1]:
!nvidia-smi

Fri Jan  1 11:51:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 36%   44C    P0    54W / 250W |      0MiB /  7981MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 29%   34C    P0    31W / 250W |      0MiB /  7982MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [2]:
from kolmov import crossval_table, get_color_fader, fit_table
import saphyra
import numpy as np
import pandas as pd
import collections
import os
import matplotlib
import matplotlib.pyplot as plt
from pprint import pprint
%config InlineBackend.figure_format = 'retina'

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


In [3]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",

              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    }
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

In [4]:
etbins = [15, 20, 30, 40, 50, 1000000]
etabins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]

## 1) Reading all tunings:


In [5]:
cv  = crossval_table( tuned_info, etbins = etbins , etabins = etabins )

In [6]:
#cv.fill(  '/Volumes/castor/tuning_data/Zee/v9/*.v9_et*.r0/*/*.gz', 'v9') # from castor external HD
cv.fill(  '/home/jodafons/public/tuning_data/Zee/v7/*.v7_et*.r0/*/*.gz', 'v7') # from LPS caloba cluster

2021-01-01 11:51:22,747 | Py.crossval_table                       INFO Reading file for v7 tag from /home/jodafons/public/tuning_data/Zee/v7/*.v7_et*.r0/*/*.gz
2021-01-01 11:51:22,748 | Py.crossval_table                       INFO There are 2500 files for this task...
2021-01-01 11:51:22,748 | Py.crossval_table                       INFO Filling the table... 
2021-01-01 11:51:48,071 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


### 1.1) Get best inits and sorts:

In [7]:
best_inits = cv.filter_inits("max_sp_val")
best_inits=best_inits.loc[best_inits.model_idx==3]
best_sorts = cv.filter_sorts( best_inits , 'max_sp_op')

### 1.2) Get best models:

Get all best models for each bin. Expected to be 25 models.

In [8]:
best_models = cv.get_best_models(best_sorts, remove_last=True)

In [9]:
best_models[0][0]['model'].summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_layer_input (InputLaye [(None, 100)]             0         
_________________________________________________________________
dense_layer (Dense)          (None, 5)                 505       
_________________________________________________________________
output_for_inference (Dense) (None, 1)                 6         
Total params: 511
Trainable params: 511
Non-trainable params: 0
_________________________________________________________________


## 2) Linear correction:

Here we will set all thresholds to operate as the same pd reference from cut-based using the pileup linear correction strategy. As the classifier efficiency has some "dependence" w.r.t the pileup we adopt the linear adjustment to "fix" the trigger efficiency. Here we will "fix" the neural network threshold w.r.t the pileup. 

### 2.1) Get all PD/Fas values:

Read all reference values from the storage.

In [10]:
# calculate all pd/fa from reference file
#ref_path = '/Volumes/castor/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'
ref_path = '/home/jodafons/public/cern_data/files/Zee/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97/references/data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.bkg.VProbes_EGAM7.GRL_v97_et{ET}_eta{ETA}.ref.pic.gz'

ref_paths = [[ ref_path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]
ref_matrix = [[ {} for eta in range(5)] for et in range(5)]
references = ['tight_cutbased', 'medium_cutbased' , 'loose_cutbased', 'vloose_cutbased']
from saphyra.core import ReferenceReader
for et_bin in range(5):
    for eta_bin in range(5):
        for name in references:
            refObj = ReferenceReader().load(ref_paths[et_bin][eta_bin])
            pd = refObj.getSgnPassed(name)/refObj.getSgnTotal(name)
            fa = refObj.getBkgPassed(name)/refObj.getBkgTotal(name)
            ref_matrix[et_bin][eta_bin][name] = {'pd':pd, 'fa':fa}

### 2.2) Create data generator:

Since each tuning models is fed by a different data organization, we need to create a generator to open the data file, prepare the matrix and apply some pre-processing (if needed).

In [11]:
def generator( path ):
    def norm1( data ):
        norms = np.abs( data.sum(axis=1) )
        norms[norms==0] = 1
        return data/norms[:,None]
    from Gaugi import load
    d = load(path)
    feature_names = d['features'].tolist()

    # How many events?
    n = d['data'].shape[0]
    
    # extract rings
    data_rings = norm1(d['data'][:,1:101])
    target = d['target']
    avgmu = d['data'][:,0]
    
    return [data_rings], target, avgmu

In [12]:
#path = '/Volumes/castor/cern_data/files/Zee/data16_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.GRL_v97.bkg.Truth.JF17/data16_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.GRL_v97.bkg.Truth.JF17_et{ET}_eta{ETA}.npz'

path = '/home/jodafons/public/cern_data/files/Zee/data16_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM1.GRL_v97.bkg.Truth.JF17/data16_13TeV.AllPeriods.sgn.probes_lhmedium.GRL_v97.bkg.Truth.JF17_et{ET}_eta{ETA}.npz'

paths = [[ path.format(ET=et,ETA=eta) for eta in range(5)] for et in range(5)]

In [13]:
# create the table class
ct  = fit_table( generator, etbins , etabins, 0.02, 0.5, 16, 40, xmin_percentage=0.05, xmax_percentage=99.95 )

### 2.3) Apply linear correction:

**NOTE**: Take about 25 minutes.

In [ ]:
# Fill it
ct.fill(paths, best_models, ref_matrix,'correction_v7_data16_13TeV_EGAM1_probes_lhmedium_JF17_Truth')


Applying ATLAS style settings...


In [ ]:
ct.table().head()

### 2.3) Create beamer report:

In [18]:
ct.dump_beamer_table(ct.table(), best_models, 'data16_13TeV v7 tuning', 
                                              'correction_v7_data16_13TeV_EGAM1_probes_lhmedium_mc15_JF17_Truth.pdf')

2021-01-01 11:43:17,946 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file correction_v7_data16_13TeV_EGAM1_probes_lhmedium_mc15_JF17_Truth.pdf latex code...


rm: cannot remove '*.aux': No such file or directory
rm: cannot remove '*.out': No such file or directory
rm: cannot remove '*.snm': No such file or directory
rm: cannot remove '*.toc': No such file or directory
rm: cannot remove '*.nav': No such file or directory


## 3) Export all tunings:

In [ ]:
model_name_format = 'mc15_13TeV.sgn.probes_lhmedium_Zee.bkg.Truth.JF17.model_v7.electron{op}.et%d_eta%d'
config_name_format = 'ElectronRinger{op}TriggerConfig.conf'
for idx, op in enumerate(['Tight','Medium','Loose','VeryLoose']):
    ct.export(best_models, 
              model_name_format.format(op=op), 
              config_name_format.format(op=op), 
              references[idx], 
              to_onnx=True)